# Data Science Essentials: Data Cleaning
    <Name> Dallin Stewart 
    <Class> ACME 002
    <Date> "[My hat] is lucky. I ain't never died while wearing that hat." 
    

In [115]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import statsmodels.formula.api as smf

### Problem 1

The g\_t\_results.csv file is a set of parent-reported scores on their child's Gifted and Talented tests. 
The two tests, OLSAT and NNAT, are used by NYC to determine if children are qualified for gifted programs.
The OLSAT Verbal has 16 questions for Kindergardeners and 30 questions for first, second, and third graders.
The NNAT has 48 questions. Each test assigns 1 point to each question asked (so there are no non integer scores).
Using this dataset, answer the following questions.



1) What column has the highest number of null values and what percent of its values are null? Print the answer as a tuple with (column name, percentage). Make sure the second value is a percent.

2) List the columns that should be numeric that aren't. Print the answer as a tuple.

3) How many third graders have scores outside the valid range for the OLSAT Verbal Score? Print the answer

4) How many data values are missing (NaN)? Print the number.


In [49]:
# load data
results = pd.read_csv('/mnt/c/Users/dalli/source/acme_senior/vl3labs/DataCleaning/g_t_results.csv')

# part 1: get the column with the most null values
nulls = results.isnull().sum()
highest = nulls.sort_values(ascending=False).head(1)
print("Column with the most null values: ")
print((highest.index[0], str(round(highest[0] / len(results) * 100, 2)) + "%"))

# part 2: show the nonnumeric columns
print("\nColumns that should be numeric but aren't: ")
print(("OLSAT Verbal Score", "OLSAT Verbal Percentile", "NNAT Non Verbal Raw Score"))

# part 3: find invalid third grader scores
third = results[results["Entering Grade Level"] == "3"]
third.loc[:, "OLSAT Verbal Score"] = pd.to_numeric(third["OLSAT Verbal Score"], errors='coerce')
third = third[(0 <= third["OLSAT Verbal Score"]) & (third["OLSAT Verbal Score"] <= 30)]
print("\nNumber of third graders' scores outside of range: ")
print(len(third))

# part 4: find the total number of nans
print("\nTotal number of nans:")
print(nulls.sum())


Column with the most null values: 
('School Assigned', '76.07%')

Columns that should be numeric but aren't: 
('OLSAT Verbal Score', 'OLSAT Verbal Percentile', 'NNAT Non Verbal Raw Score')

Number of third graders' scores outside of range: 
1

Total number of nans:
193


### Problem 2

imdb.csv contains a small set of information about 99 movies. Clean the data set by doing the following in order: 

1) Remove duplicate rows by dropping the first **or** last. Print the shape of the dataframe after removing the rows.

2) Drop all rows that contain missing data. Print the shape of the dataframe after removing the rows.

3) Remove rows that have data outside valid data ranges and explain briefly how you determined your ranges for each column.

4) Identify and drop columns with three or fewer different values. Print a tuple with the names of the columns dropped.

5) Convert the titles to all lower case.

Print the first five rows of your dataframe.

In [68]:
# load data
imdb = pd.read_csv('/mnt/c/Users/dalli/source/acme_senior/vl3labs/DataCleaning/imdb.csv')

# remove duplicate rows
imdb = imdb.drop_duplicates()
print("Part 1:")
print(imdb.shape)

# drop all rows with missing data
imdb = imdb.dropna()
print("\nPart 2:")
print(imdb.shape)

# print the max and min of each column
imdb = imdb[imdb["duration"] > 30]
imdb = imdb[imdb["title_year"] > 2000]
imdb = imdb[imdb["imdb_score"] > 0]
print("\nPart 3:")
print("It is impossible for a movie to have a duration, or imdb score less than 0, or for the movie to come out before movies were invented.")

# count the number of unique values in each column and drop if <= 3 unique values
imdb = imdb.drop(columns=["color", "language"])
print("\nPart 4:")
print(("color", "language"))

# convert titles to lowercase
imdb.loc[:, "movie_title"] = imdb["movie_title"].str.lower()
print("\nPart 5:")


Part 1:
(93, 13)

Part 2:
(64, 13)

Part 3:
It is impossible for a movie to have a duration, or imdb score less than 0, or for the movie to come out before movies were invented.

Part 4:
('color', 'language')

Part 5:


In [62]:
imdb.head()

,director_name,duration,gross,genres,movie_title,title_year,country,budget,imdb_score,actors,movie_facebook_likes
0,Martin Scorsese,240,116866727.0,Biography|Comedy|Crime|Drama,the wolf of wall street,2013,USA,100000000.0,8.2,"Leonardo DiCaprio,Matthew McConaughey,Jon Favreau",138000
1,Shane Black,195,408992272.0,Action|Adventure|Sci-Fi,iron man 3,2013,USA,200000000.0,7.2,"Robert Downey Jr.,Jon Favreau,Don Cheadle",95000
2,Quentin Tarantino,187,54116191.0,Crime|Drama|Mystery|Thriller|Western,the hateful eight,2015,USA,44000000.0,7.9,"Craig Stark,Jennifer Jason Leigh,Zoë Bell",114000
3,Kenneth Lonergan,186,46495.0,Drama,margaret,2011,usa,14000000.0,6.5,"Matt Damon,Kieran Culkin,John Gallagher Jr.",0
4,Peter Jackson,186,258355354.0,Adventure|Fantasy,the hobbit: the desolation of smaug,2013,USA,225000000.0,7.9,"Aidan Turner,Adam Brown,James Nesbitt",83000


### Problem 3
Load housing.csv into a dataframe with index=0. Descriptions of the features are in housing_data_description.txt for your convenience.  
The goal is to construct a regression model that predicts SalePrice using the other features of the dataset.  Do this as follows:


<small>
	1) Identify and handle the missing data.  Hint: Dropping every row with some missing data is not a good choice because it gives you an empty dataframe.  What can you do instead?

    2) Add two new features: 
		a) Remodeled: Whether or not a house has been remodeled with a Y if it has been
		   remodeled, or a N if it has not.
		
		b) TotalPorch: Using the 5 different porch/deck columns, create a new column that
		   provides the total square footage of all the decks and porches for each house.
    
	3) Identify the variable with nonnumeric values that are misencoded as numbers.  One-hot encode it. Hint: don't forget to remove one of the encoded columns to prevent collinearity with the constant column (which you will add later).
    
    4) Add a constant column to the dataframe.

    5) Save a copy of the dataframe.

	6) Choose four categorical featrues that seem very important in predicting SalePrice. One-hot encode these features and remove all other categorical features.
		
	7) Run an OLS using all numerical data regression on your model.  
</small>
	
Print the ten features that have the highest coef in your model. Then print the summary. Don't print the OLS

In [134]:
# load data
housing1 = pd.read_csv('/mnt/c/Users/dalli/source/acme_senior/vl3labs/DataCleaning/housing.csv')

# hot deck LotFrontage
housing1["LotFrontage"] = housing1["LotFrontage"].fillna(housing["LotFrontage"].mean())

# drop bad columns and rows with nans
housing1 = housing1.drop(columns=["Alley", "PoolQC", "Fence", "MiscFeature", "MasVnrType", "FireplaceQu"])
housing1 = housing1.dropna()
print(len(housing))

# add remodeled column
housing1["Remodeled"] = housing1["YearBuilt"] != housing1["YearRemodAdd"]
housing1["Remodeled"] = housing1["Remodeled"].replace({True: "Y", False: "N"})

# add TotalPorch column
housing1["TotalPorch"] = housing1["OpenPorchSF"] + housing1["EnclosedPorch"] + housing1["3SsnPorch"] + housing1["ScreenPorch"] + housing1["WoodDeckSF"]

# one hot encode MSSubClass, OverallQual, OverallCond
housing1 = pd.get_dummies(housing1, columns=["MSSubClass"], drop_first=True)
housingl = pd.get_dummies(housing1, columns=["OverallQual", "OverallCond"], drop_first=True)


# add a constant column to the dataframe
housing1 = sm.add_constant(housing1)

# make a copy of the dataframe
housing2 = housing1.copy()

# remove most categorical columns
housing1 = housing1.drop(columns=["MSZoning", 
                                "Street", 
                                "LotShape", 
                                "LandContour", 
                                "Utilities", 
                                "LotConfig", 
                                "LandSlope", 
                                "Neighborhood",
                                "Condition2", 
                                "HouseStyle", 
                                "RoofStyle", 
                                "RoofMatl", 
                                "Exterior1st", 
                                "Exterior2nd", 
                                "ExterQual", 
                                "ExterCond", 
                                "Foundation", 
                                "BsmtQual", 
                                "BsmtCond", 
                                "BsmtExposure", 
                                "BsmtFinType1", 
                                "BsmtFinType2", 
                                "Heating", 
                                "HeatingQC", 
                                "CentralAir", 
                                "Electrical", 
                                "KitchenQual", 
                                "Functional", 
                                "GarageType", 
                                "GarageFinish", 
                                "GarageQual", 
                                "GarageCond", 
                                "PavedDrive", 
                                "SaleType", 
                                "SaleCondition"])
housing1 = pd.get_dummies(housing1, columns=["BldgType", "Remodeled", "Condition1"], drop_first=True)
# cast all columns to np.asarray(data)
housing1 = housing1.astype(np.float64)

1338


In [135]:
# run OLS
y = housing1["SalePrice"]
X = housing1.drop(columns=["SalePrice"])
price_prediction = sm.OLS(y, X).fit()
results_as_html = price_prediction.summary().tables[1].as_html()
result_df = pd.read_html(results_as_html, header=0, index_col=0)[0]

# print the ten most important features in reverse order
print(result_df.sort_values(by="coef", ascending=False).head(10))

# print results
print(price_prediction.summary())

                       coef    std err       t  P>|t|     [0.025     0.975]
OverallQual      18270.0000   1311.021  13.932  0.000  15700.000  20800.000
GarageCars       14210.0000   3055.307   4.650  0.000   8213.871  20200.000
Condition1_Norm  11930.0000   5700.227   2.094  0.036    750.740  23100.000
BldgType_2fmCon  11110.0000  34600.000   0.321  0.749 -56800.000  79100.000
BsmtFullBath      7601.7524   2755.666   2.759  0.006   2195.621  13000.000
Remodeled_Y       6051.1051   2463.731   2.456  0.014   1217.700  10900.000
TotRmsAbvGrd      5975.0450   1302.122   4.589  0.000   3420.511   8529.579
OverallCond       5797.0538   1158.553   5.004  0.000   3524.177   8069.931
Condition1_RRNn   5479.7981  16600.000   0.330  0.742 -27100.000  38100.000
Condition1_RRAn   5209.6991   8955.418   0.582  0.561 -12400.000  22800.000
                            OLS Regression Results                            
Dep. Variable:              SalePrice   R-squared:                       0.820
Model:

### Problem 4

Using the copy of the dataframe you created in Problem 3, one-hot encode all the categorical variables.
Print the shape of the dataframe and run OLS.

Print the ten features that have the highest coef in your model and the summary.
Write a couple of sentences discussing which model is better and why.

In [136]:
# one hot encode all categorical columns of housing2
housing2 = pd.get_dummies(housing2, columns=["MSZoning", 
                                "Street", 
                                "LotShape", 
                                "LandContour", 
                                "Utilities", 
                                "LotConfig", 
                                "LandSlope", 
                                "Neighborhood",
                                "Condition2", 
                                "HouseStyle", 
                                "RoofStyle", 
                                "RoofMatl", 
                                "Exterior1st", 
                                "Exterior2nd", 
                                "ExterQual", 
                                "ExterCond", 
                                "Foundation", 
                                "BsmtQual", 
                                "BsmtCond", 
                                "BsmtExposure", 
                                "BsmtFinType1", 
                                "BsmtFinType2", 
                                "Heating", 
                                "HeatingQC", 
                                "CentralAir", 
                                "Electrical", 
                                "KitchenQual", 
                                "Functional", 
                                "GarageType", 
                                "GarageFinish", 
                                "GarageQual", 
                                "GarageCond", 
                                "PavedDrive", 
                                "SaleType", 
                                "SaleCondition",
                                "BldgType", 
                                "Remodeled", 
                                "Condition1"], drop_first=True)

# cast all columns to np.asarray(data)
housing2 = housing2.astype(np.float64)

# print the shape
print(housing2.shape)

# run OLS
y = housing2["SalePrice"]
X = housing2.drop(columns=["SalePrice"])
price_prediction = sm.OLS(y, X).fit()
results_as_html = price_prediction.summary().tables[1].as_html()
result_df = pd.read_html(results_as_html, header=0, index_col=0)[0]

# print the ten most important features in reverse order
print(result_df.sort_values(by="coef", ascending=False).head(10))

# print results
print(price_prediction.summary())

(1338, 242)
                      coef  std err       t  P>|t|    [0.025    0.975]
RoofMatl_Membran  782500.0  50300.0  15.565  0.000  684000.0  881000.0
RoofMatl_Metal    759400.0  49000.0  15.494  0.000  663000.0  856000.0
RoofMatl_WdShngl  733300.0  36500.0  20.073  0.000  662000.0  805000.0
RoofMatl_Tar&Grv  689200.0  40200.0  17.136  0.000  610000.0  768000.0
RoofMatl_Roll     685300.0  44600.0  15.380  0.000  598000.0  773000.0
RoofMatl_CompShg  679900.0  35400.0  19.207  0.000  610000.0  749000.0
RoofMatl_WdShake  673000.0  39300.0  17.141  0.000  596000.0  750000.0
GarageCond_TA     114400.0  36100.0   3.171  0.002   43600.0  185000.0
GarageCond_Gd     112900.0  37800.0   2.990  0.003   38800.0  187000.0
GarageCond_Fa     111400.0  36400.0   3.059  0.002   39900.0  183000.0
                            OLS Regression Results                            
Dep. Variable:              SalePrice   R-squared:                       0.929
Model:                            OLS   Adj. R-sq

The R squared value indicated that the second model performs better. However, each model has trade-offs between model complexity and predictive performance. The second model offers a more comprehensive representation of the data by considering a broader range of categorical factorss, and could capture more nuanced relationships and variations within these features. However, this can also lead tooverfitting, especially if the additional features introduce noise. The first model might be more less prone to overfitting, and easier to interpret, and provide more robust testing performance.
A validation dataset or cross-validation techniques would be the best comparison metic. A model with a better trade-off between accuracy and complexity is generally preferred. 